In [ ]:
* 빠른 입력

import io, os, sys
input = io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

# 스트링 처리 시 input().decode().rstrip()

import sys
input = lambda: sys.stdin.readline().rstrip()



* 깊은 재귀시 이용

sys.setrecursionlimit(100000)

from types import GeneratorType

def bootstrap(f, stack=[]):
    def wrappedfunc(*args, **kwargs):
        if stack:
            return f(*args, **kwargs)
        else:
            to = f(*args, **kwargs)
            while True:
                if type(to) is GeneratorType:
                    stack.append(to)
                    to = next(to)
                else:
                    stack.pop()
                    if not stack:
                        break
                    to = stack[-1].send(to)
            return to
    return wrappedfunc
    
    
    
* 약수 구하기

def get_divisors(num):
    divisors = set()
    for mod in range(1, int(num ** 0.5) + 1):
        if num % mod == 0:
            divisors.add(mod)
            divisors.add(num // mod)
            
    return sorted(divisors)



* 에라토스테네스의 체

max_num = 100
ip = [True] * (max_num + 1)
ip[0] = ip[1] = False
for mod1 in range(2, int((max_num + 1) ** 0.5) + 1):
    if ip[mod1]:
        for v in range(mod1 * mod1, max_num + 1, mod1):
            ip[v] = False
            
primes = [i for i, v in enumerate(ip) if v]

print(primes)



* 오일러의 체

max_num = 100
sp = [0] * (max_num + 1)
primes = []
for i in range(2, max_num + 1):
    if sp[i] == 0:
        sp[i] = i
        primes.append(i)
        
    j = 0
    while j < len(primes) and i * primes[j] <= max_num and primes[j] <= sp[i]:
        sp[i * primes[j]] = primes[j]
        j += 1
        
print(primes)



* 오일러의 체를 이용한 소인수분해

num = 12
r = []
while num > 1:
    r.append(sp[num])
    num //= sp[num]
    
print(r)



* 기본 소인수분해

def p_facto(num):
    r = []
    for mod in range(2, int(num ** 0.5) + 1):
        while num % mod == 0:
            num //= mod
            r.append(mod)
    
    if num != 1:
        r.append(num)
        
    return r

p_facto(8)



* nCk

def nCk(n, k):
    if n < k:
        return 0
    if k < 0:
        return 0
    
    if n - k < k:
        k = n - k
        
    num = 1
    for i in range(n, n - k, -1):
        num *= i
    
    denom = 1
    for i in range(2, k + 1):
        denom *= i
        
    return num // denom

r = nCk(5, 3)
print(r)



* 모듈러 곱셈 역원 nCk

mod = 1000000007
max_num = 1000

factos = [0] * (max_num + 1)
factos[0] = 1
invs = [0] * (max_num + 1)
cur = 1
for i in range(1, max_num + 1):
    cur *= i
    cur %= mod
    factos[i] = cur
    
invs[max_num] = pow(factos[max_num], mod - 2, mod)
for i in range(max_num - 1, -1, -1):
    invs[i] = (i + 1) * invs[i + 1] % mod

n, k = map(int, input().split())

def nCk(n, k):
    if n < k:
        return 0
    if k < 0:
        return 0
    return factos[n] * invs[k] * invs[n - k] % mod

r = nCk(n, k)
    
print(r)



* dp nCk

max_n = 5
max_k = 4

c_dp = [[0] * (max_k + 1) for _ in range(max_n + 1)]
for nn in range(max_n + 1):
    c_dp[nn][0] = 1
    cur = 1
    for kk in range(1, max_k + 1):
        if kk > nn:
            break
        cur = cur * (nn - (kk - 1)) // kk
        c_dp[nn][kk] = cur

nCk = lambda n, k: c_dp[n][k]

r = nCk(3, 2)



* 다익스트라

from heapq import heappush, heappop

inf = float('inf')

dists = [inf] * (n + 1)
dists[start] = 0

heap = []
heappush(heap, (0, start))

while heap:
    dist, value = heappop(heap)
    
    if dist > dists[value]:
        continue
        
    for v, d in li[value]:
        ndist = dist + d
        if ndist < dists[v]:
            dists[v] = ndist
            heappush(heap, (ndist, v))
            
print(dists)



* 플로이드 워셜

inf = float('inf')

nli = [[inf] * n for _ in range(n)]
for i in range(n):
    nli[i][i] = 0
    
for mid in range(n):
    for i in range(n):
        for j in range(n):
            nli[i][j] = min(nli[i][j], nli[i][mid] + nli[mid][j])
            
for v in nli:
    print(v)
    
    
    
* 삼분 탐색

def get_value(mid):
    cur = 0
    sv = 0
    for i in range(1, n):
        cur += mid
        sv += abs(li[i] - cur)
        
    return sv

n = int(input())
li = list(map(int, input().split()))

start = 1
end = 1000000000
result = float('inf')
while start <= end:
    mid1 = (start * 2 + end) // 3
    mid2 = (start + end * 2) // 3
    
    t1 = get_value(mid1)
    t2 = get_value(mid2)
    
    result = min(result, t1, t2)
    
    if end - start <= 5:
        for mid in range(start, end + 1):
            result = min(result, get_value(mid))
        break
    
    if t1 >= t2:
        start = mid1
    else:
        end = mid2
        
print(result)



* Sorted multiset

# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans
    
ms = SortedMultiset()
ms.add(1)
ms.index(1)
ms.index_right(1)



* Sorted Set

# https://github.com/tatyam-prime/SortedSet/blob/main/SortedSet.py
import math
from bisect import bisect_left, bisect_right
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedSet(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedSet from iterable. / O(N) if sorted and unique / O(N log N)"
        a = list(a)
        if not all(a[i] < a[i + 1] for i in range(len(a) - 1)):
            a = sorted(set(a))
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedSet" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def add(self, x: T) -> bool:
        "Add an element and return True if added. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return True
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i != len(a) and a[i] == x: return False
        a.insert(i, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()
        return True

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True
    
    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError
    
    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return 
    
    
    
* 세그먼트 트리

class SegmentTree:
    def __init__(self, data, default=10**15, func=lambda a, b: max(a,b)):
        """initialize the segment tree with data"""
        self._default = default
        self._func = func
        self._len = len(data)
        self._size = _size = 1 << (self._len - 1).bit_length()
 
        self.data = [default] * (2 * _size)
        self.data[_size:_size + self._len] = data
        for i in reversed(range(_size)):
            self.data[i] = func(self.data[i + i], self.data[i + i + 1])
 
    def __delitem__(self, idx):
        self[idx] = self._default
 
    def __getitem__(self, idx):
        return self.data[idx + self._size]
 
    def __setitem__(self, idx, value):
        idx += self._size
        self.data[idx] = value
        idx >>= 1
        while idx:
            self.data[idx] = self._func(self.data[2 * idx], self.data[2 * idx + 1])
            idx >>= 1
 
    def __len__(self):
        return self._len
 
    def query(self, start, stop):
        if start == stop:
            return self.__getitem__(start)
        stop += 1
        start += self._size
        stop += self._size
 
        res = self._default
        while start < stop:
            if start & 1:
                res = self._func(res, self.data[start])
                start += 1
            if stop & 1:
                stop -= 1
                res = self._func(res, self.data[stop])
            start >>= 1
            stop >>= 1
        return res
 
    def __repr__(self):
        return "SegmentTree({0})".format(self.data)
    
func = lambda a, b: a * b
segtree = SegmentTree(li, -inf, func=func) # 두 번째 인자는 세그트리 배열의 기본 값
segtree.query(l, r)



* 펜웍 트리

def get_sum(i):
    total = 0
    while i > 0:
        total += tree[i]
        i -= i & (-i)
        
    return total

def update(i, diff):
    while i < n + 1:
        tree[i] += diff
        i += i & (-i)

n, m = map(int, input().split())

li = [0] * (n + 1)
tree = [0] * (n + 1)

# 합계시
sv = get_sum(b) - get_sum(a - 1) # 누적합과 비슷

# 변경시
diff = nv - li[index]
li[index] = nv
update(index + 1, diff)

* LIS (nlogn)
from bisect import bisect_left

n = int(input())
li = list(map(int, input().split()))

nli = [li[0]]
for i in range(1, len(li)):
    index = bisect_left(nli, li[i])
    
    if index == len(nli):
        nli.append(li[i])
    else:
        nli[index] = li[i]
        
print(len(nli))

* Counting Inversions

def merge_sort(li):
    global count
    
    if len(li) == 1:
        return li
    
    left = merge_sort(li[:len(li) // 2])
    right = merge_sort(li[len(li) // 2:])
    
    nl = len(left)
    nr = len(right)
    
    nli = []
    l_i = r_i = 0
    while True:
        if left[l_i] <= right[r_i]:
            nli.append(left[l_i])
            count += r_i
            l_i += 1
            
            if l_i == nl:
                while r_i < nr:
                    nli.append(right[r_i])
                    r_i += 1
                break    
        else:
            nli.append(right[r_i])
            r_i += 1
            
            if r_i == nr:
                while l_i < nl:
                    nli.append(left[l_i])
                    count += r_i
                    l_i += 1
                break
    return nli 
    
count = 0
merge_sort(li)

print(count)



* CCW

def ccw(x1, y1, x2, y2, x3, y3):
    temp = x1 * y2 + x2 * y3 + x3 * y1 - x2 * y1 - x3 * y2 - x1 * y3
    
    if temp < 0:
        return -1
    elif temp > 0:
        return 1
    else:
        return 0
    
    
    
* 교란 순열

mod = 1000000000

dp = [0] * 1000001
dp[:4] = [1, 0, 1, 2]
for i in range(4, 1000001):
    dp[i] = (dp[i - 2] + dp[i - 1]) * (i - 1) % mod
    
n = int(input())

print(dp[n])



* 커스텀 기준 정렬

import functools

def compare(u, v):
    ua, ub = u
    va, vb = v
    
    r1 = va * ub
    r2 = ua * vb
    
    if ua == 0:
        return 1
    elif va == 0:
        return -1
    elif ub == vb == 0:
        if ua > va:
            return 1
        else:
            return -1
    else:
        return r1 - r2
    
li.sort(key=functools.cmp_to_key(compare))



* 분할 정복을 이용한 행렬 거듭제곱

def recur(n, size):
    global init_mul_mat
    
    if n == 1:
        return init_mul_mat
    
    mat = recur(n // 2, size)
    
    mul_mat = [[0] * size for _ in range(size)]
    for i in range(size):
        for j in range(size):
            temp = 0
            for k in range(size):
                temp += mat[i][k] * mat[k][j]
                temp %= mod
            mul_mat[i][j] = temp
            
    if n % 2 == 1:
        nmul_mat = [[0] * size for _ in range(size)]
        for i in range(size):
            for j in range(size):
                temp = 0
                for k in range(size):
                    temp += mul_mat[i][k] * init_mul_mat[k][j]
                    temp %= mod
                nmul_mat[i][j] = temp
                
        return nmul_mat
    else:
        return mul_mat
    
    
    
* 이분 매칭

import sys
# input = lambda: sys.stdin.readline().rstrip()

n, m = map(int, input().split())
possibles = [[] for _ in range(n + 1)]
for i in range(n):
    temp = list(map(int, input().split()))
    for j in range(1, temp[0] + 1):
        possibles[i + 1].append(temp[j])

count = 0
assigned = [0] * (m + 1)

def dfs(worker):
    for work in possibles[worker]:
        if not visited[work]:
            visited[work] = True
            if not assigned[work]:
                assigned[work] = worker
                return True
            if dfs(assigned[work]):
                assigned[work] = worker
                return True
    return False

for worker in range(1, n + 1):
    visited = [False] * (m + 1)
    if dfs(worker):
        count += 1
        
print(count)



* 최대 유량

from collections import deque
import sys, os, io
input = io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

inf = float('inf')

def bfs():
    levels = [-1] * (n + m + 2)
    levels[source] = 0
    
    queue = deque()
    queue.append(source)
    
    while queue:
        value = queue.popleft()
        
        for v in graph[value]:
            rem = c[value][v] - f[value][v]
            if levels[v] == -1 and rem > 0:
                levels[v] = levels[value] + 1
                
                if v != sink:
                    queue.append(v)
                
    return levels
    
def dfs(value, can_f):
    if value == sink:
        return can_f
    
    total_out_flow = 0
    for v in graph[value]:
        if levels[v] == levels[value] + 1 and f[value][v] < c[value][v]:
            rem = c[value][v] - f[value][v]
            n_can_f = min(can_f, rem)
            
            to_flow = dfs(v, n_can_f)
            f[value][v] += to_flow
            f[v][value] -= to_flow
            
            total_out_flow += to_flow
            can_f -= to_flow
            
    return total_out_flow

n, m = map(int, input().split())
A = list(map(int, input().split()))
B = list(map(int, input().split()))

c = [[0] * (n + m + 2) for _ in range(n + m + 2)]
f = [[0] * (n + m + 2) for _ in range(n + m + 2)]
graph = [[] for _ in range(n + m + 2)]
for b_num in range(m):
    temp = list(map(int, input().split()))
    
    for i in range(n):
        a_num = i + m
            
        graph[a_num].append(b_num)
        graph[b_num].append(a_num)
        c[b_num][a_num] = temp[i]
        
source = n + m
sink = n + m + 1

for b_num in range(m):
    graph[source].append(b_num)
    graph[b_num].append(source)
    c[source][b_num] = B[b_num]
    
for a_num in range(m, m + n):
    graph[a_num].append(sink)
    graph[sink].append(a_num)
    c[a_num][sink] = A[a_num - m]

total = 0
while True:
    levels = bfs()
    
    if levels[sink] == -1:
        break
    
    to_flow = dfs(source, inf)
    total += to_flow
    
print(total)



* C++ 기본 템플릿

#include <bits/stdc++.h>
using namespace std;

typedef long long ll;
ll inf = LLONG_MAX;

int main()
{
  ios::sync_with_stdio(0);
  cin.tie(0);
}